In [ ]:
#this is the base code for hyperparameter randomisation, this is not the same thing as grid search, this is
#for randomising the hyperparameters and changing episode count to later use statistical analysis to draw 
#conclusions


import os
import gym
import csv
import random
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import time


algorithm_name = "PPO"
environment_name = 'CartPole-v1'
number_of_loops = 10000

with open(f"{environment_name}_{algorithm_name} TESTING_AT_50_THOUSAND_LEARNING_RATE.csv", 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Iteration", "Learning Rate", "Batch Size", "Gamma", "GAE Lambda", "Mean Score"])

for i in range(number_of_loops):
    episode_count = 100

    # MODIFY HYPERPARAMETERS HERE
    policy = 'MlpPolicy'
    learning_rate = random.uniform(0.0001, 0.001)
    #n_steps = random.uniform(500, 4000)
    n_steps = 500
    batch_size = 64
    n_epochs = 10
    gamma = 0.99
    gae_lambda = 0.95
    clip_range = 0.2
    ent_coef = 0.0

    env = gym.make(environment_name)
    env = DummyVecEnv([lambda: env])
    log_path = "logs/"
    
    model = PPO(policy, env, verbose=1, tensorboard_log=log_path,
                learning_rate=learning_rate, n_steps=n_steps, batch_size=batch_size,
                n_epochs=n_epochs, gamma=gamma, gae_lambda=gae_lambda, clip_range=clip_range,
                ent_coef=ent_coef)

    start_time = time.time()
    number_of_episodes = 50000

    model.learn(total_timesteps=number_of_episodes)

    end_time = time.time()
    elapsed_time_ms = (end_time - start_time) * 1000
    elapsed_time_sec = elapsed_time_ms / 1000

    print(algorithm_name)
    print(environment_name)
    print("Training time: {:.3f} sec".format(elapsed_time_sec))

    model_name = f"{algorithm_name}_{environment_name}_{number_of_episodes} {learning_rate}"
    Algorithm_Path = os.path.join('Training', 'Saved Models','hyperparam testing', model_name)
    model.save(Algorithm_Path)

    episodes = 50
    scores = []
    for episode in range(1, episodes+1):
        obs = env.reset()
        done = False
        score = 0
        
        while not done:
            # env.render()
            action, _ = model.predict(obs) # WE ARE NOW USING OUR MODEL
            obs, reward, done, info = env.step(action)
            score += reward
    
        print('Episode:{} Score:{}'.format(episode, score))
        scores.append(score)
    
    env.close()

    mean_score = sum(scores) / len(scores)
    std_dev = (sum((score - mean_score)**2 for score in scores) / len(scores))**0.5
    print(algorithm_name)
    print(environment_name)
    print(f"Mean Score: {mean_score}")
    print(f"Standard Deviation: {std_dev}")

    # Save the results to a CSV file
    with open(f"{environment_name}_{algorithm_name} TESTING_AT_50_THOUSAND_LEARNING_RATE.csv", 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([i+1, learning_rate, batch_size, gamma, gae_lambda, mean_score])
    
    del model